In [137]:
import requests
import pandas as pd

NETWORK_URL = "https://api.citybik.es/v2/networks/bikesampa"

response = requests.get(NETWORK_URL)

In [138]:
def clean_stations(data):
    stations = []
    for station in data['network']['stations']:
        station_data = {}
        for key in station.keys():
            if key != 'extra':
                station_data[key] = station[key]
            else:
                for extra_key in station['extra'].keys():
                    station_data[extra_key] = station['extra'][extra_key]
        stations.append(station_data)
    
    return pd.DataFrame(stations)

In [139]:
def get_distances(data):
    supersim_lat = -23.556641
    supersim_lng = -46.681632
    distances = []
    for station in data['network']['stations']:
        lat = station['latitude']
        lng = station['longitude']
        p1 = (lat, lng)
        p2 = (supersim_lat, supersim_lng)
        distance = geodesic(p1, p2).km
        distances.append(distance)
    return distances

In [140]:
df_stations = clean_stations(data)

In [141]:
df_stations

,id,name,latitude,longitude,timestamp,free_bikes,empty_slots,uid,renting,returning,...,post_code,has_ebikes,ebikes,normal_bikes,payment,payment-terminal,altitude,slots,rental_uris,virtual
0,0495af0c779f79ebaa01242df456963d,204 - Consolação,-23.556727,-46.664197,2025-08-17T20:00:25.505310+00:00Z,0,19,204,1,1,...,03164200,True,0,0,"[key, transitcard, phone]",False,-23.56,19,{},False
1,05ac42b0b09d9dda95118ee7ff00d5bc,150 - Metrô Paraíso,-23.575386,-46.641375,2025-08-17T20:00:25.504968+00:00Z,0,19,150,1,1,...,NaN,True,0,0,"[key, transitcard, creditcard, phone]",True,NaN,19,{},False
2,07e1bc56a4d45c04055860ae39f9322a,7 - Praça Joaquim Roberto,-23.572544,-46.693681,2025-08-17T20:00:25.503928+00:00Z,10,9,7,1,1,...,03164200,True,0,10,"[key, transitcard, phone]",False,NaN,19,{},False
3,086f46ad981708395ea90ca6ae362832,434 - Praça Procópio Ferreira II,-23.599550,-46.690740,2025-08-17T20:00:25.506774+00:00Z,3,12,592,1,1,...,04533-085,True,0,3,"[key, transitcard, phone]",False,0.00,15,{},False
4,0b509d7429586cacb0776fd8b203d61c,443 - Cardoso Melo III,-23.596440,-46.688550,2025-08-17T20:00:25.506569+00:00Z,0,15,556,1,1,...,04548-005,True,0,0,"[key, transitcard, phone]",False,0.00,15,{},False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,fd5d8d6dcc34499ffa8ca4db591e8e45,317 - R. Leonardo Nunes,-23.598733,-46.639103,2025-08-17T20:00:25.505845+00:00Z,1,14,317,1,1,...,03164200,True,0,1,"[key, transitcard, creditcard, phone]",True,NaN,15,{},False
236,fdb890213c54ad952730bd9fab78a147,76 - 8° Batalhão,-23.574298,-46.654040,2025-08-17T20:00:25.504419+00:00Z,2,13,76,1,1,...,03164200,True,0,2,"[key, transitcard, creditcard, phone]",True,NaN,15,{},False
237,fe23e24943b116bf2b7bbfe856292ffa,131 - Ibirapuera Portão 06,-23.593685,-46.660042,2025-08-17T20:00:25.504800+00:00Z,13,6,131,1,1,...,03164200,True,0,13,"[key, transitcard, phone]",False,NaN,19,{},False
238,ff724047677a832632b2ec87805270c0,215 - E.E. Godofredo Furtado,-23.559442,-46.678456,2025-08-17T20:00:25.505348+00:00Z,1,14,215,1,1,...,NaN,True,0,1,"[key, transitcard, creditcard, phone]",True,NaN,15,{},False


In [142]:
len(get_distances(data))

240

In [143]:
df_stations['distance_to_supersim'] = get_distances(data)

In [144]:
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    240 non-null    object 
 1   name                  240 non-null    object 
 2   latitude              240 non-null    float64
 3   longitude             240 non-null    float64
 4   timestamp             240 non-null    object 
 5   free_bikes            240 non-null    int64  
 6   empty_slots           240 non-null    int64  
 7   uid                   240 non-null    object 
 8   renting               240 non-null    int64  
 9   returning             240 non-null    int64  
 10  last_updated          239 non-null    float64
 11  address               239 non-null    object 
 12  post_code             205 non-null    object 
 13  has_ebikes            240 non-null    bool   
 14  ebikes                240 non-null    int64  
 15  normal_bikes          2

In [118]:
import psycopg2

In [136]:
conn = psycopg2.connect(
    dbname="weather_db",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Insert row by row
insert_query = """
    INSERT INTO public.citybik_log (id, updated_at, station_name, free_bikes, empty_slots, address, distance_to_supersim, renting, "returning", has_ebikes, ebikes, normal_bikes, payment_terminal)
    VALUES (%s, NOW(), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT DO NOTHING;
"""

for _, row in df_stations[['id','name','free_bikes','empty_slots','address','distance_to_supersim','renting','returning','has_ebikes','ebikes','normal_bikes','payment-terminal']].iterrows():
    cur.execute(insert_query, tuple(row))

conn.commit()
cur.close()
conn.close()